In [208]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import ta
from datapackage import Package
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import os
import random
import shutil
import time

In [209]:
# Selects the desired number of random stocks so they can be used to perform tests
# Selected tables will be copied to the test_data folder, which is emptied beforehand

n_files = 10
sample_files = random.choices(os.listdir("data"), k=n_files)
for i in os.listdir("test_data"):
	os.remove("test_data/" + i)
for file in sample_files:
	shutil.copy2("data/" + file, "test_data/")

# Replicates the big_table creating process on a much smaller scale
# (also doesn't save it to file)

test_table = pd.DataFrame(columns=["Date", "Close", "Volume", "Code"])
for i in os.listdir("test_data"):
	little_table = pd.read_csv("test_data/" + i)
	little_table['Code'] = i.replace(".csv", "")
	test_table = pd.concat([test_table, little_table[["Date", "Close", "Volume", "Code"]]], axis=0)


In [210]:
# Replicates the big_table creating process on a much smaller scale
# (also doesn't save it to file)

test_table = pd.DataFrame()
for i in os.listdir("test_data"):
	little_table = pd.read_csv("test_data/" + i)
	little_table['Code'] = i.replace(".csv", "")
	little_table['Return'] = 1 - (little_table["Close"])
	test_table = pd.concat([test_table, little_table], axis=0)

d:\ProgramFiles\Anaconda\envs\final-project\lib\site-packages\ta\trend.py:730: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  self._trs[0] = diff_directional_movement.dropna()[
d:\ProgramFiles\Anaconda\envs\final-project\lib\site-packages\ta\trend.py:748: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  self._dip[0] = pos.dropna()[0: self._window].sum()
d:\ProgramFiles\Anaconda\envs\final-project\lib\site-packages\ta\trend.py:760: FutureWarning: The behavior of `series[i:j]` with an 

In [211]:
display(test_table.describe())
display(test_table.isna().sum())
display(test_table.dtypes)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,volume_adi,volume_obv,volume_cmf,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
count,56406.000000,56406.000000,56406.000000,56406.000000,5.640600e+04,56406.000000,56406.000000,4.624800e+04,5.640600e+04,55931.000000,...,56156.000000,56076.000000,56076.000000,56156.000000,56076.000000,56076.000000,56316.000000,56406.000000,56396.000000,56406.000000
mean,53.849749,54.595248,53.194897,53.909477,2.432183e+06,0.005253,0.000570,inf,4.139153e+08,0.041904,...,0.241270,0.241626,0.002156,-1.640456,-1.622055,0.019662,53.768383,0.069077,0.044811,3668.314305
std,107.845443,109.060719,106.398419,107.759991,5.031973e+06,0.449387,0.033862,NaN,5.823930e+08,0.194398,...,2.888629,2.735723,0.829813,13.232547,10.384016,7.269119,107.397099,2.822113,2.636987,8610.934265
min,0.000000,0.107967,0.105986,0.107967,0.000000e+00,0.000000,0.000000,-4.877656e+08,-4.941982e+08,-0.869557,...,-41.338818,-35.387497,-9.738605,-76.443989,-73.957602,-25.352215,0.110759,-98.725723,-56.953310,-99.309524
25%,4.178955,4.246178,4.107222,4.175309,2.664000e+05,0.000000,0.000000,4.404920e+07,6.094696e+07,-0.076259,...,-0.835209,-0.766194,-0.387049,-9.230640,-7.491530,-4.290885,4.185187,-0.961519,-0.965772,52.591426
50%,14.665817,14.860000,14.441284,14.674657,1.072925e+06,0.000000,0.000000,1.376122e+08,1.617895e+08,0.046297,...,0.462083,0.458954,-0.008795,-2.511595,-1.849694,-0.786504,14.597382,0.000000,0.000000,284.445941
75%,52.207098,52.901152,51.519348,52.223746,2.383500e+06,0.000000,0.000000,3.236394e+08,4.656882e+08,0.164794,...,1.624260,1.552704,0.367724,5.298159,4.326060,3.260940,52.060026,1.034754,1.029872,1698.081827
max,861.615302,866.056908,855.129175,863.679626,1.559718e+08,103.750000,3.000000,inf,2.279061e+09,0.946437,...,26.391993,22.727973,9.132304,94.061768,49.438417,92.390482,832.718679,54.947357,43.791524,64365.852238


Date                   0
Open                   0
High                   0
Low                    0
Close                  0
                    ... 
momentum_pvo_hist    330
momentum_kama         90
others_dr              0
others_dlr            10
others_cr              0
Length: 95, dtype: int64

Date                  object
Open                 float64
High                 float64
Low                  float64
Close                float64
                      ...   
momentum_pvo_hist    float64
momentum_kama        float64
others_dr            float64
others_dlr           float64
others_cr            float64
Length: 95, dtype: object

In [212]:
display(test_table.head())
display(test_table.tail())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Code,volume_adi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,1991-11-08 00:00:00-05:00,0.25847,0.258470,0.258470,0.258470,88028800,0.0,0.0,APH,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-98.380589,NaN,0.000000
1,1991-11-11 00:00:00-05:00,0.25847,0.258470,0.258470,0.258470,11859200,0.0,0.0,APH,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
2,1991-11-12 00:00:00-05:00,0.25847,0.258470,0.258470,0.258470,13097600,0.0,0.0,APH,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
3,1991-11-13 00:00:00-05:00,0.25847,0.258470,0.251484,0.251484,19059200,0.0,0.0,APH,-19059200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.702880,-2.740080,-2.702880
4,1991-11-14 00:00:00-05:00,0.23402,0.244499,0.234020,0.244499,20835200,0.0,0.0,APH,1776000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.777592,-2.816896,-5.405397


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Code,volume_adi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
10760,2022-11-16 00:00:00-05:00,334.369995,338.619995,333.279999,335.640015,357900,0.0,0.0,TYL,NaN,...,-1.498172,-2.957438,1.459266,9.105527,14.225357,-5.119830,308.338659,-0.462633,-0.463706,39088.102683
10761,2022-11-17 00:00:00-05:00,330.559998,330.559998,318.880005,320.549988,352000,0.0,0.0,TYL,NaN,...,-1.339881,-2.633926,1.294046,7.384183,12.857122,-5.472939,309.144686,-4.495896,-4.600097,37326.246242
10762,2022-11-18 00:00:00-05:00,326.420013,326.940002,312.059998,313.450012,407000,0.0,0.0,TYL,NaN,...,-1.378131,-2.382767,1.004636,7.067486,11.699195,-4.631709,309.383821,-2.214936,-2.239834,36497.279014
10763,2022-11-21 00:00:00-05:00,312.339996,322.489990,310.339996,321.820007,257100,0.0,0.0,TYL,NaN,...,-1.183188,-2.142851,0.959664,3.632832,10.085922,-6.453090,310.264730,2.670281,2.635251,37474.529085
10764,2022-11-22 00:00:00-05:00,323.079987,327.019989,319.649994,326.929993,283812,0.0,0.0,TYL,NaN,...,-0.889469,-1.892175,1.002705,1.384392,8.345616,-6.961224,311.387609,1.587840,1.575365,38071.152318


In [213]:
# Cleans data and 
start_time = time.time()
for i in range(len(test_table)):
	test_table["Date"].iloc[i] = test_table["Date"].iloc[i][:10]
	if (i % 1000 == 0):
		print(f"{i} records processed of {len(test_table)}")
test_table["Date"] = pd.to_datetime(test_table["Date"])
test_table["Volume"] = pd.to_numeric(test_table["Volume"])
test_table = test_table.dropna()
print(f"Process executed in {time.time() - start_time}")

C:\Users\BogPoet\AppData\Local\Temp\ipykernel_24056\103732121.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_table["Date"].iloc[i] = test_table["Date"].iloc[i][:10]


0 records processed of 56406
1000 records processed of 56406
2000 records processed of 56406
3000 records processed of 56406
4000 records processed of 56406
5000 records processed of 56406
6000 records processed of 56406
7000 records processed of 56406
8000 records processed of 56406
9000 records processed of 56406
10000 records processed of 56406
11000 records processed of 56406
12000 records processed of 56406
13000 records processed of 56406
14000 records processed of 56406
15000 records processed of 56406
16000 records processed of 56406
17000 records processed of 56406
18000 records processed of 56406
19000 records processed of 56406
20000 records processed of 56406
21000 records processed of 56406
22000 records processed of 56406
23000 records processed of 56406
24000 records processed of 56406
25000 records processed of 56406
26000 records processed of 56406
27000 records processed of 56406
28000 records processed of 56406
29000 records processed of 56406
30000 records processed

In [214]:
display(test_table.head())
display(test_table.dtypes)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Code,volume_adi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr


Date                 datetime64[ns]
Open                        float64
High                        float64
Low                         float64
Close                       float64
                          ...      
momentum_pvo_hist           float64
momentum_kama               float64
others_dr                   float64
others_dlr                  float64
others_cr                   float64
Length: 95, dtype: object

In [215]:
test_table.isna().sum()

Date                 0.0
Open                 0.0
High                 0.0
Low                  0.0
Close                0.0
                    ... 
momentum_pvo_hist    0.0
momentum_kama        0.0
others_dr            0.0
others_dlr           0.0
others_cr            0.0
Length: 95, dtype: float64

In [ ]:
df=pd.DataFrame()
df.ta.

AttributeError: 'DataFrame' object has no attribute 'pta'

In [218]:
test_table.isna().sum()

Date                 0.0
Open                 0.0
High                 0.0
Low                  0.0
Close                0.0
                    ... 
momentum_pvo_hist    0.0
momentum_kama        0.0
others_dr            0.0
others_dlr           0.0
others_cr            0.0
Length: 95, dtype: float64

In [220]:
test_table.ta.sma()

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

In [223]:
tyl = pd.DataFrame(pd.read_csv("test_data/TYL.csv"))

,ABER_ZG_20_15,ABER_SG_20_15,ABER_XG_20_15,ABER_ATR_20_15
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
10760,319.376501,330.979862,307.773140,11.603361
10761,318.735001,330.682139,306.787863,11.947138
10762,317.922834,330.065496,305.780172,12.142662
10763,316.993501,329.136652,304.850350,12.143151


In [238]:
tyl.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,a,b,c,d
0,1980-03-17 00:00:00-05:00,0.0,0.897661,0.848249,0.856484,53400,0.0,0.0,ABER_ZG_20_15,ABER_SG_20_15,ABER_XG_20_15,ABER_ATR_20_15
1,1980-03-18 00:00:00-05:00,0.0,0.840013,0.807072,0.831778,65400,0.0,0.0,ABER_ZG_20_15,ABER_SG_20_15,ABER_XG_20_15,ABER_ATR_20_15
2,1980-03-19 00:00:00-05:00,0.0,0.840013,0.807072,0.831778,28200,0.0,0.0,ABER_ZG_20_15,ABER_SG_20_15,ABER_XG_20_15,ABER_ATR_20_15
3,1980-03-20 00:00:00-05:00,0.0,0.840014,0.807072,0.807072,21000,0.0,0.0,ABER_ZG_20_15,ABER_SG_20_15,ABER_XG_20_15,ABER_ATR_20_15
4,1980-03-21 00:00:00-05:00,0.0,0.831778,0.790601,0.790601,23400,0.0,0.0,ABER_ZG_20_15,ABER_SG_20_15,ABER_XG_20_15,ABER_ATR_20_15
